In [1]:
# class InterpolET():
#     def __init__(self):

# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def formating_labvanced (lb):
    # Renaming the columns
    lb = lb[lb['Task_Name'] == "large_grid"]
    lb = lb.rename(columns={"value":"X_lb","Unnamed: 11":"Y_lb","Unnamed: 12":"time_lb","Unnamed: 13":'c'})
    lb['time_lb'] = lb['time_lb'].fillna(0)
    lb = lb[(lb[['time_lb']] != 0).all(axis=1)]
    # Converting time column to 13 digit number
    lb.time_lb = lb.time_lb.apply(lambda x: int('%.0f' % x))

    # # Fill all nan values with 0
    lb[['X_lb','Y_lb']] = lb[['X_lb','Y_lb']].fillna(0)

    # # Convert the coordinates points from floats to intergers
    lb[['X_lb','Y_lb']] = lb[['X_lb','Y_lb']].apply(np.int64)
    lb = lb[(lb[['X_lb']] != 0).all(axis=1)]
    
#     # Drop NaNs
#     lb.dropna(inplace=True)

    return lb

def formating_eyelink (el):
    el.Time = el.Time.apply(lambda x: int('%.0f' % (x * 1)))
    # el['Time'] = el['Time'].apply(np.int64) * 1000
    # EYE LINK DATA rename the eyelink columns
    el = el.rename(columns={'X':'X_el','Y':'Y_el','Time':'t'})
    el['time_el'] = el['t']
    #If so delete this row
    el = el[(el[['X_el','Y_el','time_el']] != 0).all(axis=1)]
    el.dropna(inplace=True)
    #Check once agian
    (el == 0).sum(axis=0)
    
    ## Setup the Eyelink timeseries column as an index
    el = el.set_index('t')
    return el

# Takes two parameters df of each EyeTracker
def interpolation (el, lb):
# Interpolate data to have equal index size
    
    df_temp = pd.concat([el, lb .index.to_frame()]).sort_index().interpolate()

    df_temp = df_temp[~df_temp.index.duplicated(keep='first')]
    df_interpolated = lb .merge(df_temp, left_index=True, right_index=True, how='left')

    df_interpolated = df_interpolated.drop(columns=['time_lb'])
    df_interpolated = df_interpolated.reset_index()

    df_interpolated .Y_el = df_interpolated.Y_el.astype(int)
    df_interpolated .X_el = df_interpolated.X_el.astype(int)
    return df_interpolated

